In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification
# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# Load data
train_df = pd.read_csv("/kaggle/input/mc-datathon-2025-sentiment-analysis/train.csv")
test_df = pd.read_csv("/kaggle/input/mc-datathon-2025-sentiment-analysis/test.csv")

# Drop NaN in sentiment
train_df = train_df.dropna(subset=["sentiment"])

# Encode labels (0=negative, 1=neutral, 2=positive)
label_enc = LabelEncoder()
train_df["label"] = label_enc.fit_transform(train_df["sentiment"])
print("Classes:", label_enc.classes_)

# Load locally uploaded RoBERTa model
model_path = "/kaggle/input/roberta-base/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=3).to(device)

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

# Split train/val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"].tolist(),
    train_df["label"].tolist(),
    test_size=0.1,
    stratify=train_df["label"],
    random_state=42
)

train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

# Metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    f1 = f1_score(labels, preds, average="macro")
    return {"f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none" 
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Train
trainer.train()


2025-07-23 07:45:39.323363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753256739.548464      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753256739.616004      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using: cpu
Classes: ['negative' 'neutral' 'positive']


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/roberta-base/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_36/4013610155.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=430, training_loss=0.7578576376271803, metrics={'train_runtime': 21762.7261, 'train_samples_per_second': 0.316, 'train_steps_per_second': 0.02, 'total_flos': 1807062952955904.0, 'train_loss': 0.7578576376271803, 'epoch': 1.0})

In [ ]:

# Predict on test set
test_dataset = SentimentDataset(test_df["text"].tolist())
preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

# Decode labels
decoded_preds = label_enc.inverse_transform(pred_labels)

# Submission
submission = pd.DataFrame({
    "id": test_df["id"],
    "sentiment": decoded_preds
})
submission.to_csv("submission.csv", index=False)